In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import logging
import uuid
from pathlib import Path
from datetime import datetime, timedelta
from typing import Optional, Dict, List, Tuple
from client import LocalLLMClient, create_client
from news_function import analyze_news_single  # 复用单条分析函数


# --------------------------
# 1. 基础配置
# --------------------------
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - 批量拆分分析 - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("batch_split_analysis.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# 结果存储目录（自动创建）
OUTPUT_DIR = Path("batch_split_results")
OUTPUT_DIR.mkdir(exist_ok=True)

# 定期保存的批次大小（每处理10条新闻保存一次）
CHECKPOINT_BATCH_SIZE = 10


# --------------------------
# 2. 工具函数：追加数据到Parquet文件
# --------------------------
def append_to_parquet(df: pd.DataFrame, file_path: str, schema: Optional[pa.Schema] = None):
    """
    将DataFrame追加到现有的Parquet文件
    
    参数：
    - df: 要追加的数据
    - file_path: 目标Parquet文件路径
    - schema: 数据 schema，确保一致性
    """
    table = pa.Table.from_pandas(df)
    
    # 如果文件不存在，创建新文件；否则追加
    if not Path(file_path).exists():
        # 使用当前表的schema作为初始schema
        pq.write_table(table, file_path)
    else:
        # 确保schema一致
        if schema is None:
            existing_table = pq.read_table(file_path)
            schema = existing_table.schema
        
        # 追加数据
        with pq.ParquetWriter(file_path, schema) as writer:
            writer.write_table(table)


# --------------------------
# 3. 核心函数：生成双表数据（单批次）
# --------------------------
def _process_batch(batch_results: List[Dict]) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """处理单批次结果，生成行业裁决表和个股推荐表的DataFrame片段"""
    industry_records: List[Dict] = []
    stock_records: List[Dict] = []

    for result in batch_results:
        # 基础元数据
        base_meta = {
            "original_news_id": result["original_news_id"],
            "original_news_date": result["original_news_date"],
            "original_news_title": result["original_news_title"],
            "analysis_status": result["status"],
            "error_msg": result["error_msg"],
            "log_path": result["log_path"],
            "analysis_time": result["analysis_batch_time"]
        }

        # 处理分析失败的记录
        if result["status"] == "failed":
            industry_records.append({
                **base_meta,
                "analysis_id": f"ana_{uuid.uuid4().hex[:8]}",
                "industry": "",
                "impact_direction": "",
                "confidence_score": 0,
                "comprehensive_reason": ""
            })
            continue

        # 处理分析成功的记录
        for industry in result["ruled_industries"]:
            current_analysis_id = f"ana_{uuid.uuid4().hex[:8]}"

            # 行业裁决主表记录
            industry_records.append({
                **base_meta,
                "analysis_id": current_analysis_id,
                "industry": industry.get("industry", ""),
                "impact_direction": industry.get("impact", ""),
                "confidence_score": industry.get("confidence", 0),
                "comprehensive_reason": industry.get("comprehensive_reason", "")
            })

            # 个股推荐副表记录
            for rank, stock in enumerate(industry.get("stocks", []), 1):
                stock_records.append({
                    "analysis_id": current_analysis_id,
                    "stock_name": stock.get("name", ""),
                    "stock_code": stock.get("code", ""),
                    "recommendation_reason": stock.get("reason", ""),
                    "stock_rank": rank
                })

    # 转换为DataFrame并调整字段顺序
    industry_df = pd.DataFrame(industry_records)[[
        "analysis_id", "original_news_id", "original_news_date", "original_news_title",
        "industry", "impact_direction", "confidence_score", "comprehensive_reason",
        "analysis_status", "error_msg", "log_path", "analysis_time"
    ]]

    stock_df = pd.DataFrame(stock_records)[[
        "analysis_id", "stock_name", "stock_code", "recommendation_reason", "stock_rank"
    ]]

    return industry_df, stock_df


# --------------------------
# 4. 批量分析主函数（带定期保存功能）
# --------------------------
def batch_analyze_split_parquet(
    input_parquet: str,
    news_date: str,
    debate_rounds: int = 2,
    llm_client: Optional[LocalLLMClient] = None,
    checkpoint_batch_size: int = CHECKPOINT_BATCH_SIZE,** llm_kwargs
) -> Tuple[str, str]:
    """
    批量分析Parquet新闻，带定期保存功能，防止程序崩溃导致数据丢失
    
    参数：
    - input_parquet: 输入新闻Parquet路径
    - news_date: 新闻日期
    - debate_rounds: 辩论轮次
    - llm_client: 已实例化的LLM客户端
    - checkpoint_batch_size: 每处理多少条新闻保存一次
    - **llm_kwargs: 创建LLM客户端的参数
    
    返回：
    - 行业裁决表路径, 个股推荐表路径
    """
    # 1. 初始化LLM客户端
    if not llm_client:
        try:
            llm_client = create_client(** llm_kwargs)
            logger.info("LLM客户端初始化成功")
        except Exception as e:
            raise Exception(f"LLM客户端初始化失败：{str(e)}")

    # 2. 读取并处理输入Parquet（逆序）
    try:
        logger.info(f"读取输入Parquet：{input_parquet}")
        input_df = pd.read_parquet(input_parquet)
        required_cols = ["id", "date", "title", "content"]
        missing_cols = [col for col in required_cols if col not in input_df.columns]
        if missing_cols:
            raise Exception(f"输入缺少字段：{', '.join(missing_cols)}")
        
        # 逆序处理（按新闻id降序）
        # 筛选出指定日期的新闻
        input_df_sorted = input_df[input_df["date"] == news_date].sort_values(by="id", ascending=True).reset_index(drop=True)
        total_news = len(input_df_sorted)
        logger.info(f"读取 {total_news} 条新闻，已按id逆序")
    except Exception as e:
        raise Exception(f"读取输入失败：{str(e)}")

    # 3. 准备输出文件路径（带时间戳）
    time_suffix = datetime.now().strftime("%Y%m%d_%H%M%S")
    industry_path = OUTPUT_DIR / f"industry_verdict_{time_suffix}.parquet"
    stock_path = OUTPUT_DIR / f"stock_recommendation_{time_suffix}.parquet"
    
    # 4. 批量处理新闻，定期保存
    batch_results: List[Dict] = []
    processed_count = 0
    industry_schema = None
    stock_schema = None

    for idx, row in input_df_sorted.iterrows():
        news_id = str(row["id"])
        logger.info(f"分析第 {idx+1}/{total_news} 条新闻（ID：{news_id}）")
        
        # 调用单条分析函数
        single_result = analyze_news_single(
            news_title=str(row["title"]),
            news_content=str(row["content"]),
            debate_rounds=debate_rounds,
            llm_client=llm_client
        )

        # 补充原始新闻元数据
        single_result.update({
            "original_news_id": news_id,
            "original_news_date": str(row["date"]),
            "original_news_title": str(row["title"]),
            "analysis_batch_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })
        batch_results.append(single_result)
        processed_count += 1

        # 达到批次大小，保存一次数据
        if processed_count % checkpoint_batch_size == 0 or processed_count == total_news:
            logger.info(f"已处理 {processed_count}/{total_news} 条新闻，开始保存中间结果...")
            
            # 处理当前批次结果
            batch_industry_df, batch_stock_df = _process_batch(batch_results)
            
            # 首次保存时确定schema
            if industry_schema is None and not batch_industry_df.empty:
                industry_schema = pa.Table.from_pandas(batch_industry_df).schema
            if stock_schema is None and not batch_stock_df.empty:
                stock_schema = pa.Table.from_pandas(batch_stock_df).schema
            
            # 追加保存
            append_to_parquet(batch_industry_df, str(industry_path), industry_schema)
            append_to_parquet(batch_stock_df, str(stock_path), stock_schema)
            
            # 清空批次结果列表，准备下一批
            batch_results = []
            logger.info(f"中间结果保存完成，行业裁决表：{industry_path}，个股推荐表：{stock_path}")

    logger.info(f"全部处理完成，共处理 {processed_count} 条新闻")
    return str(industry_path), str(stock_path)


# --------------------------
# 5. 使用示例
# --------------------------
if __name__ == "__main__":
    # 配置参数
    INPUT_PARQUET = "../data/stock_daily_cctvnews.parquet"  # 输入新闻路径
    DEBATE_ROUNDS = 2  # 辩论轮次
    CHECKPOINT_SIZE = 10  # 每处理10条新闻保存一次
    LLM_KWARGS = {
        # "model": "gpt-4-turbo",  # 实际使用时填写LLM参数
        # "api_key": "your_api_key"
    }
    # 设定获取数据的日期为昨日
    news_date = '2025-03-30' #(datetime.now() - timedelta(days=1)).strftime("%Y%m%d")
    try:
        # 执行批量拆分分析
        industry_table_path, stock_table_path = batch_analyze_split_parquet(
            input_parquet=INPUT_PARQUET,
            news_date=news_date,
            debate_rounds=DEBATE_ROUNDS,
            checkpoint_batch_size=CHECKPOINT_SIZE,
            **LLM_KWARGS
        )

        # 打印结果
        print("="*70)
        print("批量拆分分析完成！")
        print(f"输入文件：{INPUT_PARQUET}")
        print(f"日期：{news_date}")
        print(f"行业裁决表：{industry_table_path}")
        print(f"个股推荐表：{stock_table_path}")
        print(f"提示：程序在处理过程中已自动保存中间结果，可通过 'analysis_id' 字段关联两张表")
        print("="*70)
    except Exception as e:
        print(f"批量分析失败：{str(e)}")
        logger.error(f"批量分析失败：{str(e)}")

2025-10-11 16:40:05,740 - INFO - LLM客户端初始化成功
2025-10-11 16:40:05,740 - INFO - 读取输入Parquet：../data/stock_daily_cctvnews.parquet
2025-10-11 16:40:05,822 - INFO - 读取 16 条新闻，已按id逆序
2025-10-11 16:40:05,824 - INFO - 分析第 1/16 条新闻（ID：118222）
2025-10-11 16:40:05,825 - INFO - 开始分析单条新闻（ID：news_single_20251011164005_894，标题：习近平就朝鲜劳动党成立80周年向朝鲜劳动党总书记金正恩致贺电...）
2025-10-11 16:40:05,826 - INFO - 请求模型: qwen3:30b-a3b-instruct-2507-q4_K_M
2025-10-11 16:40:05,827 - INFO - 用户消息: 标题：习近平就朝鲜劳动党成立80周年向朝鲜劳动党总书记金正恩致贺电
内容：央视网消息
（新闻联播）：10月10日，中共中央总书记习近平致电朝鲜劳动党总书记金正恩，祝贺朝鲜劳动党成立80周年。
习近平在贺电中说，在朝鲜劳动党成立80周年之际，我谨代表中国共产党中央委员会，并以我个人名义，向总书记同志和朝鲜劳动党中央、全体朝鲜劳动党党员以及朝鲜人民致以热烈祝贺和美好祝福。
习近平表示，80年来，朝鲜劳动党团结带领朝鲜人民奋发进取、攻坚克难，推动朝鲜社会主义事业取得可喜成就。近年来，总书记同志领导朝鲜党和人民积极致力于加强党建、发展经济、改善民生。祝愿在以总书记同志为首的朝鲜劳动党坚强领导下，朝鲜社会主义事业不断取得新成就，迎接朝鲜劳动党九大胜利召开。
习近平强调，中朝两国同为共产党领导的社会主义国家。近年来，我同总书记同志多次会晤，为两党两国关系发展领航把舵，开启中朝友谊崭新篇章。前不久，总书记同志来华出席纪念中国人民抗日战争暨世界反法西斯战争胜利80周年活动，我同总书记同志深入会谈，为中朝双方进一步发展友好合作关系指明了前进方向。无论国际形势如何变化，维护好、巩固好、发展好中朝关系，始终是中国党和政府不变的方针。中方愿同朝方一道，加强战略沟通，深化务实合作，密切协